In [1]:
# import library
import pandas as pd
import re
import spacy
import nltk
from tqdm import tqdm

#Load the dataset
dataset = pd.read_csv("/Users/fuchuenli/Desktop/Data Science /Year 2/Trimester 1/COMP SCI 7417/Week 9/Assignemt 3/Corpus_10k.csv")
too_long = [index for index, text in enumerate(dataset["corpus"]) if type(text) != str or len(text) < 100 or len(text) > 100000]
dataset = dataset.loc[~dataset.index.isin(too_long)].reset_index(drop=True)

# Data cleaning
def data_cleaning(corpus):
    pattern = r"\[\d+(,\s*\d+)*\]|\(([A-Za-z]+( [A-Za-z]+)+)\., [0-9]+\)|http://\S+|www.\S+|[^\x00-\x7F]+"
    clean_corpus = re.sub(pattern, "", corpus)
    return clean_corpus
dataset["corpus"] = dataset["corpus"].apply(lambda x: data_cleaning(x))

# Load the coreferenced corpus
for index in range(len(dataset)):
    with open(f"/Users/fuchuenli/Desktop/Data Science /Year 2/Trimester 1/COMP SCI 7417/Week 9/Assignemt 3/Coreference_10k/corpus_coref_{index}.txt", "r") as f:
        dataset["corpus"][index] = f.read()
# A function to the corpus paragraphs by graphs
def paragraph_tokenize(text):
    return nltk.tokenize.blankline_tokenize(text)
dataset["corpus"] = dataset["corpus"].apply(lambda x: paragraph_tokenize(x))

corpus = dataset["corpus"].tolist()

# Spacy pipeline
nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe("merge_entities")
print('Pipeline components included: ', nlp.pipe_names)
# Pass the corpus to spacy pipeline
for article_index in tqdm(range(len(corpus))):
    for fragment_index in range(len(corpus[article_index])):
        corpus[article_index][fragment_index] = nlp(corpus[article_index][fragment_index])

/Users/fuchuenli/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_sci_sm' (0.5.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.4.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Pipeline components included:  ['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer', 'parser', 'ner', 'merge_noun_chunks', 'merge_entities']


100%|██████████| 9786/9786 [2:43:46<00:00,  1.00s/it]  


In [3]:
# Export the processed corpus 
import pickle
with open('corpus_pickle_10k_paragraph', 'wb') as f:
    pickle.dump(corpus, f)